# Clustering Neighborhood - Toronto

In [7]:
# import modules
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

!pip install geocoder
# Libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# Tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


!pip install folium==0.5.0
import folium # Plotting library

# Matplotlib modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Library SKlearn for clustering
from sklearn.cluster import KMeans


# Extracting Data From Wikipedia page of "List of postal codes of Canada"

In [8]:
from bs4 import BeautifulSoup #To retrive data from website using BeautifulSoup
toronto_neighbor_page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(toronto_neighbor_page, 'html.parser')
allTable = soup.find('table')
allElement = allTable.find_all('td')

In [9]:
#Extract postal code, borough and neighborhood
postalCode = []
borough = []
neighborhood = []
counter = 1
for element in allElement[0:]:
    textHere = element.text[0:-1]
    if counter == 1:
        counter = counter + 1
        postalCode.append(textHere)
    elif counter == 2:
        counter = counter + 1
        borough.append(textHere)
    else: 
        counter = 1
        neighborhood.append(textHere)
# print(postalCode)
# print(borough)
# print(neighborhood)

In [32]:
# Constructing a dataframe
df = pd.DataFrame({'PostalCode':postalCode,'Borough':borough,'Neighborhood':neighborhood})
df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [33]:
# perform postal code check
postal_codes = df['PostalCode']
postal_codes

0      M1A
1      M2A
2      M3A
3      M4A
4      M5A
      ... 
175    M5Z
176    M6Z
177    M7Z
178    M8Z
179    M9Z
Name: PostalCode, Length: 180, dtype: object

In [41]:
# data obtained
df.shape
df['Borough'].value_counts()
df=df[(df['Borough']!='Not assigned')]
postal_codes = df['PostalCode']
postal_codes
df.shape #the size of the cleaned data
df

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


# Finding the latitude and longitude

In [13]:
import geocoder
# test geolocator for toronto
geolocator = Nominatim(user_agent="My_App")

location = geolocator.geocode('{}, Toronto, ON'.format('M3A'))

location

# define function to retrieve latitude and longitude from postal code
def get_latlon(postal_code):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

get_latlon('M9B') # test function

[43.65034000000003, -79.55361999999997]

In [42]:
coord = {}
for code in postal_codes:
    loc = get_latlon(code)
    coord[code] =loc 


In [43]:
latitude=[]
longitude=[]
for code in postal_codes:
    latitude.append(coord[code][0])
    longitude.append(coord[code][1])
df['Latitude']=latitude
df['Longitude']=longitude
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.75245,-79.32991
3,M4A,North York,Victoria Village,43.73057,-79.31306
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
...,...,...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.65319,-79.51113
165,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.64869,-79.38544
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.63278,-79.48945


In [49]:
df.reset_index(drop=True, inplace=True)
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.65319,-79.51113
99,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.64869,-79.38544
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.63278,-79.48945
